In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Difference between BSE and SE images

In [6]:
import pathlib
import PIL 
import numpy
from fastai.vision.all import *

input_path = Path('./data')

def get_items(input_path, file_type):
    file_names = get_image_files(input_path)
    file_names = [filename for filename in file_names if file_type in str( filename ) ]
    return file_names

def statistics( filename ):
    with PIL.Image.open(filename) as image:
        data = numpy.array( image )
        return ( numpy.average(data), numpy.std(data) )

file_names = get_items(input_path, "_bse")    

accumulated = []

for bse_filename in file_names:
    bse_filename = str(bse_filename)
    bse_avg,bse_std = statistics(bse_filename)
    se_filename = bse_filename.replace("_bse", "_se")
    se_avg,se_std = statistics(se_filename)
    # print( bse_filename )    
    # print( "bse avg", bse_avg, "bse std", bse_std, "se avg", se_avg, "se std", se_std )    
    std_ratio = bse_std / se_std
    accumulated = accumulated + std_ratio
    
accumulated = accumulated / len(file_names)
    
print( "average ratio of std dev", std_ratio )    


average ratio of std dev 1.691678199401348


## Noise level in BSE images

In [14]:
from scipy import ndimage

input_path = Path('./data')

all_bse_mean = 0.0
all_se_mean = 0.0

all_bse_std = 0.0
all_se_std = 0.0

n_items = 0

file_names = get_items(input_path, "_bse")
for bse_filename in file_names:
    bse_filename = str(bse_filename)
    se_filename = bse_filename.replace("_bse", "_se")
    gt_filename = bse_filename.replace("_bse", "_gt")
    # print(bse_filename, gt_filename)
    
    with PIL.Image.open(gt_filename) as gt_image:
         with PIL.Image.open(bse_filename) as bse_image:
             with PIL.Image.open(se_filename) as se_image:
                gt_data  = numpy.array( gt_image )                
                gt_data = ndimage.binary_erosion( gt_data, iterations=3 )                
                bse_data = numpy.array( bse_image )
                se_data  = numpy.array( se_image )
                masked_bse = bse_data[ gt_data == 1 ]
                masked_se  =  se_data[ gt_data == 1 ]
                all_bse_std = all_bse_std + numpy.std(masked_bse)
                all_se_std  = all_se_std + numpy.std(masked_se)
                
                all_bse_mean = all_bse_mean + numpy.mean( bse_image )
                all_se_mean = all_se_mean + numpy.mean( se_image )
                n_items = n_items + 1
            
all_bse_std = all_bse_std / float(n_items)
all_se_std = all_se_std / float(n_items)           
all_bse_mean = all_bse_mean / float(n_items)
all_se_mean = all_se_mean / float(n_items)

print("all bse std", all_bse_std)
print("all  se std", all_se_std)

print("all bse SNR", 128.0 / all_bse_std)
print("all  se SNR", 128.0 / all_se_std)

all bse std 13.260512933774717
all  se std 10.463907502565428
all bse SNR 9.652718611961244
all  se SNR 12.23252403259665
